In [ ]:
%defaultDatasource jdbc:h2:mem:db

# Etapa 4 - atualização de queries SQL

In [ ]:
CREATE TABLE Country (
    iso_code_3c VARCHAR(8) NOT NULL,
    name VARCHAR(50),
    population DOUBLE,
    population_density DOUBLE,
    median_age DOUBLE,
    aged_65_older DOUBLE,
    aged_70_older DOUBLE,
    gdp_per_capita DOUBLE,
    extreme_poverty DOUBLE,
    cardiovasc_death_rate DOUBLE,
    diabetes_prevalence DOUBLE,
    female_smokers DOUBLE,
    male_smokers DOUBLE,
    handwashing_facilities DOUBLE,
    hospital_beds_per_thousand DOUBLE,
    life_expectancy DOUBLE,
    human_development_index DOUBLE,
    PRIMARY KEY (iso_code_3c)
) AS SELECT
    iso_code,
    location,
    population,
    population_density,
    median_age,
    aged_65_older,
    aged_70_older,
    gdp_per_capita,
    extreme_poverty,
    cardiovasc_death_rate,
    diabetes_prevalence,
    female_smokers,
    male_smokers,
    handwashing_facilities,
    hospital_beds_per_thousand,
    life_expectancy,
    human_development_index
    FROM CSVREAD('data/owid-country-data.csv');

CREATE TABLE Continent (
    code VARCHAR (2) NOT NULL,
    name varchar (20) NOT NULL,
    PRIMARY KEY (CODE)
) AS SELECT
    Code,
    Name
    FROM CSVREAD('data/continent-codes_csv.csv');

CREATE TABLE _country_temp (
    iso_code_2c VARCHAR(2),
    iso_code_3c VARCHAR(3),
    continent_code VARCHAR(2),
    country_number INTEGER,
) AS SELECT 
    Two_Letter_Country_Code,
    Three_Letter_Country_Code,
    Country_Number
    FROM CSVREAD('data/country-and-continent-codes-list-csv_csv.csv');

CREATE TABLE CovidCases (
    iso_code VARCHAR(8) NOT NULL,
    date DATE NOT NULL,
    total_cases DOUBLE,
    new_cases DOUBLE, 
    total_deaths DOUBLE,
    new_deaths DOUBLE,
    total_tests DOUBLE,
    FOREIGN KEY (iso_code) REFERENCES Country (iso_code)
) AS SELECT
    iso_code,
    date,
    total_cases,
    new_cases,
    total_deaths,
    new_deaths,
    total_tests
    FROM CSVREAD('data/owid-covid-data.csv');

CREATE TABLE CountryBorders (
    iso_code_2c VARCHAR (2) NOT NULL,
    country_border_iso_code_2c VARCHAR(2) NOT NULL,
    FOREIGN KEY (iso_code_2c) REFERENCES Country (iso_code_2c),
    FOREIGN KEY (country_border_iso_code_2c) REFERENCES Country (iso_code)
) AS SELECT
    country_code,
    country_border_code,
    FROM CSVREAD('data/GEODATASOURCE-COUNTRY-BORDERS.csv');

In [ ]:
SELECT * FROM Country;
SELECT * FROM CovidCases;
SELECT * FROM Continent;
SELECT * FROM _country_temp;
SELECT * FROM CountryBorders;

# Queries atualizadas com base em nossa nova análise junto às fontes de dados adicionadas

## 1) Update da tabela Country para adicionar o continente e sigla de 2 letras

In [ ]:
ALTER TABLE Country ADD COLUMN iso_code_2c VARCHAR(2);
ALTER TABLE Country ADD COLUMN continent_code VARCHAR(2);
ALTER TABLE Country ADD COLUMN country_number INTEGER;

UPDATE Country c
JOIN _country_temp temp ON c.iso_code_3c = temp.iso_code_3c
SET 
    c.iso_code_2c       = temp.iso_code_2c,
    c.continent_code    = temp.continent_code,
    c.country_number    = temp.country_number;

ALTER TABLE Country 
ADD FOREIGN KEY (continent_code) REFERENCES Continent(code);



## 2) Número de mortes por continente

In [ ]:
SELECT co.name as Continente, SUM(cv.total_deaths) as Total_Mortes, COUNT(c.iso_code_3c) Paises
FROM Country c, CovidCases cv, Continent co
WHERE co.code = c.continent_code, c.iso_code_3c = cv.iso_code_3c
GROUP BY co.name
ORDER BY Total_Mortes DESC

## 3) Quantidade de fronteiras que cada país possui

In [ ]:
CREATE VIEW QtdadeFronteiras AS
SELECT cb.iso_code_2c, Count(*) AS Quantidade
    FROM CountryBorders cb
    GROUP BY cs.iso_code_2c;

SELECT *
    FROM QtdadeFronteiras
    ORDER BY DESC

SELECT c.name as Pais, COUNT()

## 4) Número de mortes por COVID por ordem de IDH

In [ ]:
SELECT c.name, co.total_deaths, c.human_development_index, co.date
    FROM Country c, CasosCovid co
    WHERE c.iso_code = co.iso_code
    ORDER BY co.date DESC, c.human_development_index DESC
    LIMIT 210

## 5) Análise da espectativa de vida com mortes por COVID

In [ ]:
SELECT c.name, c.life_expectancy, co.total_deaths, co.date
    FROM Country c, CasosCovid co
    WHERE c.iso_code = co.iso_code
    ORDER BY co.date DESC, c.life_expectancy DESC
    LIMIT 210